<a href="https://colab.research.google.com/github/Andrew-Sams/Building-Purchase-Analysis/blob/main/Building_Purchase_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Pip installs
!pip install numpy pandas seaborn matplotlib scipy
!pip install openpyxl
!pip install yfinance
!pip install numpy_financial
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00


In [18]:
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Function to calculate monthly mortgage payment
def calculate_mortgage(principal, annual_interest_rate, years=30):
    monthly_interest_rate = annual_interest_rate / 12
    num_payments = years * 12
    mortgage_payment = principal * (monthly_interest_rate * (1 + monthly_interest_rate)**num_payments) / ((1 + monthly_interest_rate)**num_payments - 1)
    return mortgage_payment

# Function to run simulations with savings check
def run_simulations_with_savings_check(purchase_price, savings, annual_base_income_range, annual_base_expense_range, interest_rate_range, closing_cost_percentage_range, additional_upfront_costs_range, additional_annual_income_range, additional_annual_costs_range, property_growth_rate_range, inflation_rate_range, years, target_irr):
    favorable_outcomes = 0
    total_simulations = 1_000
    irrs = []
    count_irr_above_target = 0

    for _ in range(total_simulations):
        # Random values within ranges
        income = np.random.uniform(*annual_base_income_range)
        expense = np.random.uniform(*annual_base_expense_range)
        interest_rate = np.random.uniform(*interest_rate_range)
        closing_cost_percentage = np.random.uniform(*closing_cost_percentage_range)
        additional_upfront_costs = np.random.uniform(*additional_upfront_costs_range)
        additional_annual_income = np.random.uniform(*additional_annual_income_range)
        additional_annual_costs = np.random.uniform(*additional_annual_costs_range)
        property_growth_rate = np.random.uniform(*property_growth_rate_range)
        inflation_rate = np.random.uniform(*inflation_rate_range)

        # Initial calculations
        down_payment = purchase_price * down_payment_percentage
        loan_amount = purchase_price - down_payment
        closing_costs = loan_amount * closing_cost_percentage
        initial_outlay = down_payment + closing_costs + additional_upfront_costs

        # Check if savings are sufficient for initial outlay
        if savings < initial_outlay:
            continue  # Skip simulation as it's unfavorable

        cash_flows = [-initial_outlay]
        debt = 0

        monthly_mortgage = calculate_mortgage(loan_amount, interest_rate)
        annual_mortgage_payment = monthly_mortgage * 12

        for year in range(1, years + 1):
            # Adjust for inflation
            income *= (1 + inflation_rate)
            expense *= (1 + inflation_rate)
            additional_annual_income *= (1 + inflation_rate)
            additional_annual_costs *= (1 + inflation_rate)

            # Annual cash flow
            annual_cash_flow = income + additional_annual_income - expense - annual_mortgage_payment - additional_annual_costs

            # Managing debt if cash flow is negative
            if annual_cash_flow < 0:
                debt += abs(annual_cash_flow)  # Adding negative cash flow to debt
                debt *= 1.10  # Applying 10% interest on the debt
            else:
                # If there's enough cash flow to pay off the debt
                if annual_cash_flow > debt:
                    annual_cash_flow -= debt  # Paying off the debt
                    debt = 0
                else:
                    debt -= annual_cash_flow  # Paying as much as possible
                    annual_cash_flow = 0  # All cash flow goes to debt repayment

            cash_flows.append(annual_cash_flow)

        # Final year sale price calculation with closing costs
        gross_sale_price = purchase_price * (1 + property_growth_rate)**years
        sale_closing_costs = gross_sale_price * 0.06
        final_sale_price = gross_sale_price - sale_closing_costs
        cash_flows[-1] += final_sale_price

        # Calculate IRR
        try:
            irr = npf.irr(cash_flows)
        except ValueError:
            continue

        if not np.isnan(irr) and not np.isinf(irr):
            irrs.append(irr)
            if irr > target_irr:
                count_irr_above_target += 1

        if annual_cash_flow >= 0:
            favorable_outcomes += 1

    favorable_percentage = favorable_outcomes / total_simulations * 100
    average_irr = np.mean(irrs) if irrs else None
    percent_above_target_irr = count_irr_above_target / len(irrs) * 100 if irrs else 0

    return favorable_percentage, average_irr, percent_above_target_irr

# Function to update and display plots
def update_plots(savings_amount, interest_rate_range, down_payment_percentage, closing_cost_percentage_range, additional_upfront_costs_range, annual_base_income_range, annual_base_expense_range, additional_annual_income_range, additional_annual_costs_range, property_growth_rate_range, inflation_rate_range, years, target_irr):
    # Running simulations for this specific savings amount
    purchase_prices = np.arange(1_000_000, 4_100_000, 100_000)
    results_for_savings = [run_simulations_with_savings_check(
        price,
        savings_amount,
        annual_base_income_range,
        annual_base_expense_range,
        interest_rate_range,
        closing_cost_percentage_range,
        additional_upfront_costs_range,
        additional_annual_income_range,
        additional_annual_costs_range,
        property_growth_rate_range,
        inflation_rate_range,
        years,
        target_irr
    ) for price in purchase_prices]

    # Separating the results
    favorable_percentages, average_irrs, percentages_above_target = zip(*results_for_savings)

    from scipy.interpolate import interp1d
    import matplotlib.colors as mcolors

    # Plotting the results for the specific savings amount
    plt.figure(figsize=(10, 16))

    # Interpolating the data points for a smoother curve
    interpolator = interp1d(purchase_prices, favorable_percentages, kind='linear')
    fine_purchase_prices = np.linspace(purchase_prices.min(), purchase_prices.max(), 500)
    fine_favorable_percentages = interpolator(fine_purchase_prices)

    # Find the interpolated purchase price closest to 50% favorable outcomes
    target_percentage = 50
    closest_interp_index = np.abs(fine_favorable_percentages - target_percentage).argmin()
    closest_interp_price = fine_purchase_prices[closest_interp_index]

    # Define two separate color maps for each side of the target price
    left_cmap = mcolors.LinearSegmentedColormap.from_list("", ["green", "green", "yellow"])
    right_cmap = mcolors.LinearSegmentedColormap.from_list("", ["red", "red", "red", "red", "red"])

    # Plot for Favorable Percentages with gradient shading
    plt.subplot(3, 1, 1)
    plt.plot(fine_purchase_prices, fine_favorable_percentages, label='Favorable Percentages')

    # Apply gradient shading on both sides of the target price
    for i in range(len(fine_purchase_prices) - 1):
        if fine_purchase_prices[i] <= closest_interp_price:
            color = left_cmap(i / closest_interp_index)
        else:
            color = right_cmap((i - closest_interp_index) / (len(fine_purchase_prices) - closest_interp_index))
        plt.fill_between(fine_purchase_prices[i:i+2], fine_favorable_percentages[i:i+2], color=color, alpha=0.1)

    # Annotating the interpolated 50% point
    plt.annotate(f'${closest_interp_price / 1e6:.2f} million (50% point)', (closest_interp_price, target_percentage),
                textcoords="offset points", xytext=(0,10), ha='center', arrowprops=dict(arrowstyle='->'))

    plt.xlabel('Purchase Price ($)')
    plt.ylabel('Percentage Favorable (%)')
    plt.title('Percentage Affording Upfront & On-going Costs')

    # Setting y-axis to display every 10th percentile
    plt.yticks(np.arange(0, 101, 10));

    # Interpolating the IRR data points
    interpolator_irr = interp1d(purchase_prices, percentages_above_target, kind='linear')
    fine_irrs = interpolator_irr(fine_purchase_prices)

    # Find the interpolated purchase price closest to target IRR percentage
    closest_irr_index = np.abs(fine_irrs - target_percentage).argmin()
    closest_irr_price = fine_purchase_prices[closest_irr_index]

    plt.subplot(3, 1, 2)
    plt.plot(fine_purchase_prices, fine_irrs, label='Percentages Above 6.5% IRR')

    # Apply gradient shading on both sides of the target IRR price
    for i in range(len(fine_purchase_prices) - 1):
        if fine_purchase_prices[i] <= closest_irr_price:
            color = left_cmap(i / closest_irr_index)
        else:
            color = right_cmap((i - closest_irr_index) / (len(fine_purchase_prices) - closest_irr_index))
        plt.fill_between(fine_purchase_prices[i:i+2], fine_irrs[i:i+2], color=color, alpha=0.1)

    # Annotating the interpolated target IRR point
    plt.annotate(f'${closest_irr_price / 1e6:.2f} million (50% Point)', (closest_irr_price, target_percentage),
                textcoords="offset points", xytext=(0,10), ha='center', arrowprops=dict(arrowstyle='->'))

    plt.xlabel('Purchase Price ($)')
    plt.ylabel('Percentage Above 6.5% IRR (%)')
    plt.title('Percentage Above 6.5% IRR vs. Purchase Price')

    # Setting y-axis to display every 10th percentile
    plt.yticks(np.arange(0, 101, 10));

# Function to be called when button is clicked
def on_button_clicked(b):
    with output:
        output.clear_output(wait=True)  # Clear the previous plots before drawing new ones
        # Now call the function to draw the plots
        update_plots(
            savings_amount_widget.value,
            interest_rate_widget.value,
            down_payment_widget.value,
            closing_cost_percentage_widget.value,
            additional_upfront_costs_widget.value,
            annual_base_income_widget.value,
            annual_base_expense_widget.value,
            additional_annual_income_widget.value,
            additional_annual_costs_widget.value,
            property_growth_rate_widget.value,
            inflation_rate_widget.value,
            years_widget.value,
            target_irr_widget.value
        )
        plt.show()  # Make sure to call plt.show() to display the new plots

# Create widgets
savings_amount_widget = widgets.FloatSlider(value=600000, min=100000, max=1000000, step=10000, description='Savings Amount:')
interest_rate_widget = widgets.FloatRangeSlider(value=[0.07, 0.08], min=0, max=0.1, step=0.01, description='Interest Rate Range:')
down_payment_widget = widgets.FloatSlider(value=0.2, min=0.1, max=0.3, step=0.01, description='Down Payment %:')
closing_cost_percentage_widget = widgets.FloatRangeSlider(value=[0.04, 0.07], min=0, max=0.1, step=0.01, description='Closing Cost % Range:')
additional_upfront_costs_widget = widgets.FloatRangeSlider(value=[0, 50000], min=0, max=100000, step=5000, description='Additional Upfront Costs Range:')
annual_base_income_widget = widgets.FloatRangeSlider(value=[250000, 350000], min=100000, max=500000, step=10000, description='Annual Base Income Range:')
annual_base_expense_widget = widgets.FloatRangeSlider(value=[80000, 150000], min=50000, max=200000, step=10000, description='Annual Base Expense Range:')
additional_annual_income_widget = widgets.FloatRangeSlider(value=[0, 50000], min=0, max=100000, step=5000, description='Additional Annual Income Range:')
additional_annual_costs_widget = widgets.FloatRangeSlider(value=[20000, 80000], min=10000, max=100000, step=5000, description='Additional Annual Costs Range:')
property_growth_rate_widget = widgets.FloatRangeSlider(value=[-0.04, 0.06], min=-0.1, max=0.1, step=0.01, description='Property Growth Rate Range:')
inflation_rate_widget = widgets.FloatRangeSlider(value=[0, 0.04], min=0, max=0.1, step=0.01, description='Inflation Rate Range:')
years_widget = widgets.IntSlider(value=20, min=5, max=30, step=1, description='Years:')
target_irr_widget = widgets.FloatSlider(value=0.065, min=0.05, max=0.1, step=0.005, description='Target IRR:')

# Adjust layout for widgets to prevent title overlap
widget_list = [
    savings_amount_widget,
    interest_rate_widget,
    down_payment_widget,
    closing_cost_percentage_widget,
    additional_upfront_costs_widget,
    annual_base_income_widget,
    annual_base_expense_widget,
    additional_annual_income_widget,
    additional_annual_costs_widget,
    property_growth_rate_widget,
    inflation_rate_widget,
    years_widget,
    target_irr_widget
]

# Set a specific description width that fits the longest description
description_width = 'initial'  # You can also set a specific value like '300px'

for widget in widget_list:
    widget.style = {'description_width': description_width}
    widget.layout = widgets.Layout(width='100%')  # Use the full width available

# Create a button for triggering updates
button = widgets.Button(description="Update Plots")
button.on_click(on_button_clicked)

# Create an output area for the plots
output = widgets.Output()

# Display widgets and button
display(savings_amount_widget,
    interest_rate_widget,
    down_payment_widget,
    closing_cost_percentage_widget,
    additional_upfront_costs_widget,
    annual_base_income_widget,
    annual_base_expense_widget,
    additional_annual_income_widget,
    additional_annual_costs_widget,
    property_growth_rate_widget,
    inflation_rate_widget,
    years_widget,
    target_irr_widget)  # Add all widgets here
display(button, output)

FloatSlider(value=600000.0, description='Savings Amount:', layout=Layout(width='100%'), max=1000000.0, min=100…

FloatRangeSlider(value=(0.07, 0.08), description='Interest Rate Range:', layout=Layout(width='100%'), max=0.1,…

FloatSlider(value=0.2, description='Down Payment %:', layout=Layout(width='100%'), max=0.3, min=0.1, step=0.01…

FloatRangeSlider(value=(0.04, 0.07), description='Closing Cost % Range:', layout=Layout(width='100%'), max=0.1…

FloatRangeSlider(value=(0.0, 50000.0), description='Additional Upfront Costs Range:', layout=Layout(width='100…

...

FloatRangeSlider(value=(0.0, 0.04), description='Inflation Rate Range:', layout=Layout(width='100%'), max=0.1,…

IntSlider(value=20, description='Years:', layout=Layout(width='100%'), max=30, min=5, style=SliderStyle(descri…

FloatSlider(value=0.065, description='Target IRR:', layout=Layout(width='100%'), max=0.1, min=0.05, step=0.005…

Button(description='Update Plots', style=ButtonStyle())

Output()